In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/predict-future-sales-translated-dataset/shops_en.csv
/kaggle/input/predict-future-sales-translated-dataset/items_en.csv
/kaggle/input/predict-future-sales-translated-dataset/item_categories_en.csv
/kaggle/input/competitive-data-science-predict-future-sales/items.csv
/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv
/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv
/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv
/kaggle/input/competitive-data-science-predict-future-sales/shops.csv
/kaggle/input/competitive-data-science-predict-future-sales/test.csv


## 1. 라이브러리 추가

In [2]:
import datetime #시계열 데이터 처리
import numpy as np
import pandas as pd

#시각화 라이브러리
import seaborn as sns
import matplotlib.pyplot as plt

#전처리 라이브러리
from sklearn.preprocessing import StandardScaler, MinMaxScaler

#모델 라이브러리
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from xgboost import plot_importance #xgboost 중요 피처 시각화

#성능 측정
from sklearn.metrics import mean_squared_error

## 2. 데이터 셋 가져오기

In [3]:
train = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv')
test = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/test.csv')
item = pd.read_csv('/kaggle/input/predict-future-sales-translated-dataset/items_en.csv')
item_cat = pd.read_csv('/kaggle/input/predict-future-sales-translated-dataset/item_categories_en.csv')
shop = pd.read_csv('/kaggle/input/predict-future-sales-translated-dataset/shops_en.csv')

sample = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv')

### 2.1 데이터 병합

In [4]:
merge_item = pd.merge(train,item, on = 'item_id', how = 'left')
merge_item_cat = pd.merge(merge_item ,item_cat, on = 'item_category_id', how = 'left')
merge_shop = pd.merge(merge_item_cat, shop, on ='shop_id', how = 'left')
#merge_shop = merge_shop.reset_index(['date','date_block_num','shop_id','shop_name'])
data = merge_shop.copy()

In [5]:
data.head(10)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_name,item_category_id,item_category_name,shop_name
0,02.01.2013,0,59,22154,999.00,1.0,Scene 2012 (BD),37,Movies - Blu-Ray,"Yaroslavl shopping center ""Altair"""
1,03.01.2013,0,25,2552,899.00,1.0,DEEP PURPLE The House Of Blue Light LP,58,Music - Vinyl,"Moscow TRC ""Atrium"""
2,05.01.2013,0,25,2552,899.00,-1.0,DEEP PURPLE The House Of Blue Light LP,58,Music - Vinyl,"Moscow TRC ""Atrium"""
3,06.01.2013,0,25,2554,1709.05,1.0,DEEP PURPLE Who Do You Think We Are LP,58,Music - Vinyl,"Moscow TRC ""Atrium"""
4,15.01.2013,0,25,2555,1099.00,1.0,DEEP PURPLE 30 Very Best Of 2CD (Фирм.),56,Music - CD production firm,"Moscow TRC ""Atrium"""
5,10.01.2013,0,25,2564,349.00,1.0,DEEP PURPLE Perihelion: Live In Concert DVD (К...,59,Music - Music video,"Moscow TRC ""Atrium"""
6,02.01.2013,0,25,2565,549.00,1.0,DEEP PURPLE Stormbringer (фирм.),56,Music - CD production firm,"Moscow TRC ""Atrium"""
7,04.01.2013,0,25,2572,239.00,1.0,In premonition of love I and F S,55,Music - CD of local production,"Moscow TRC ""Atrium"""
8,11.01.2013,0,25,2572,299.00,1.0,In premonition of love I and F S,55,Music - CD of local production,"Moscow TRC ""Atrium"""
9,03.01.2013,0,25,2573,299.00,3.0,DEL REY LANA Born To Die,55,Music - CD of local production,"Moscow TRC ""Atrium"""


## 3. EDA

### 3.0 기본 정보 확인

In [6]:
#기본 정보
data.head(10)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_name,item_category_id,item_category_name,shop_name
0,02.01.2013,0,59,22154,999.00,1.0,Scene 2012 (BD),37,Movies - Blu-Ray,"Yaroslavl shopping center ""Altair"""
1,03.01.2013,0,25,2552,899.00,1.0,DEEP PURPLE The House Of Blue Light LP,58,Music - Vinyl,"Moscow TRC ""Atrium"""
2,05.01.2013,0,25,2552,899.00,-1.0,DEEP PURPLE The House Of Blue Light LP,58,Music - Vinyl,"Moscow TRC ""Atrium"""
3,06.01.2013,0,25,2554,1709.05,1.0,DEEP PURPLE Who Do You Think We Are LP,58,Music - Vinyl,"Moscow TRC ""Atrium"""
4,15.01.2013,0,25,2555,1099.00,1.0,DEEP PURPLE 30 Very Best Of 2CD (Фирм.),56,Music - CD production firm,"Moscow TRC ""Atrium"""
5,10.01.2013,0,25,2564,349.00,1.0,DEEP PURPLE Perihelion: Live In Concert DVD (К...,59,Music - Music video,"Moscow TRC ""Atrium"""
6,02.01.2013,0,25,2565,549.00,1.0,DEEP PURPLE Stormbringer (фирм.),56,Music - CD production firm,"Moscow TRC ""Atrium"""
7,04.01.2013,0,25,2572,239.00,1.0,In premonition of love I and F S,55,Music - CD of local production,"Moscow TRC ""Atrium"""
8,11.01.2013,0,25,2572,299.00,1.0,In premonition of love I and F S,55,Music - CD of local production,"Moscow TRC ""Atrium"""
9,03.01.2013,0,25,2573,299.00,3.0,DEL REY LANA Born To Die,55,Music - CD of local production,"Moscow TRC ""Atrium"""


In [7]:
#데이터 셋 요약
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2935849 entries, 0 to 2935848
Data columns (total 10 columns):
 #   Column              Dtype  
---  ------              -----  
 0   date                object 
 1   date_block_num      int64  
 2   shop_id             int64  
 3   item_id             int64  
 4   item_price          float64
 5   item_cnt_day        float64
 6   item_name           object 
 7   item_category_id    int64  
 8   item_category_name  object 
 9   shop_name           object 
dtypes: float64(2), int64(4), object(4)
memory usage: 246.4+ MB


In [8]:
#수치 데이터 요약
data.describe()

,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_category_id
count,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06
mean,1.456991e+01,3.300173e+01,1.019723e+04,8.908532e+02,1.242641e+00,4.000138e+01
std,9.422988e+00,1.622697e+01,6.324297e+03,1.729800e+03,2.618834e+00,1.710076e+01
min,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,-2.200000e+01,0.000000e+00
25%,7.000000e+00,2.200000e+01,4.476000e+03,2.490000e+02,1.000000e+00,2.800000e+01
50%,1.400000e+01,3.100000e+01,9.343000e+03,3.990000e+02,1.000000e+00,4.000000e+01
75%,2.300000e+01,4.700000e+01,1.568400e+04,9.990000e+02,1.000000e+00,5.500000e+01
max,3.300000e+01,5.900000e+01,2.216900e+04,3.079800e+05,2.169000e+03,8.300000e+01


### 3.1 결측치 확인

In [9]:
data.isna().sum()

date                  0
date_block_num        0
shop_id               0
item_id               0
item_price            0
item_cnt_day          0
item_name             0
item_category_id      0
item_category_name    0
shop_name             0
dtype: int64

### 3.2 Shop 이름이 같은 Shop 처리

In [10]:
#테스트 셋에 없는 shop_id 
set(data['shop_id'].unique()) - set(test['shop_id'].unique()) 

{0, 1, 8, 9, 11, 13, 17, 20, 23, 27, 29, 30, 32, 33, 40, 43, 51, 54}

In [11]:
#전체 shop 이름
shop['shop_name']

0                  ! Yakutsk Ordzhonikidze, 56 Franc
1                       ! Yakutsk TC "Central" Franc
2                                   Adygea TC "Mega"
3                   Balashikha TRC "October-Kinomir"
4                         Volzhsky mall "Volga Mall"
5                            Vologda SEC "Marmalade"
6                      Voronezh (Plekhanovskaya, 13)
7                            Voronezh TRC "Maksimir"
8                      Voronezh TRC City-Park "City"
9                                    Itinerant trade
10                    Zhukovsky Street. Chkalov 39m?
11                    Zhukovsky Street. Chkalov 39m²
12                           Shop Online Emergencies
13                   Kazan shopping center "Bahetle"
14          Kazan shopping mall of "Parkhouse" II of
15                          Kaluga SEC "XXI Century"
16                                  Kolomna TC "Rio"
17                    Krasnoyarsk TC "Vzletka Plaza"
18                             Krasnoyarsk SC 

In [12]:
#이름이 겹치는 shop 종류 : (1 58)(10 11)(30 31 32)(39 40)
def shop_diff_cat(shop_id1,shop_id2):
    shop_1 = data.loc[data['shop_id']==shop_id1,'shop_name'].unique()
    shop_2 = data.loc[data['shop_id']==shop_id2,'shop_name'].unique()
    data1 = data[data['shop_id'].isin([shop_id1])].loc[:,'item_category_name'].unique()
    data2 = data[data['shop_id'].isin([shop_id2])].loc[:,'item_category_name'].unique()

    data1_set = set(list(data1))
    data2_set = set(list(data2))
    dup_set = data2_set - data1_set #shop의 중복 Id 중복 데이터 셋
    sort_list = sorted(list(dup_set))
    print(f'shop1 이름: {shop_1}\nshop2 이름: {shop_2}')
    print("중복되지 않은 카테고리" )
    for cat in sort_list:
        print(cat)

In [13]:
shop_diff_cat(1,58)

shop1 이름: ['! Yakutsk TC "Central" Franc']
shop2 이름: ['Yakutsk TC "Central"']
중복되지 않은 카테고리
Accessories - XBOX ONE
Books - Comics, Manga
Books - artbook, encyclopedia
Game consoles - Other
Game consoles - PS4
Game consoles - XBOX ONE
Games - PS4
Gifts - Attributes
Gifts - Bags, Albums, Mats d / mouse
Movie - Blu-Ray 4K
Music - Gift Edition
Music - MP3
Music - Music video
Payment card (Movies, Music, Games)
System Tools
Tickets (digits)
Игры - XBOX ONE


### 3.3 item_category_name(big), city, building col 생성

In [14]:
# - (hypen) 붙은 문장 앞이 대분류
#카테고리(대분류)생성
data['item_category_name(big)'] = data['item_category_name'].map(lambda x : x.split('-')[0].strip())
#도시,복합쇼핑몰 column 생성
data['city'] = data['shop_name'].map(lambda x : x.split(' ')[0])
data['buildings'] = data['shop_name'].map(lambda x : x.split(' ')[1])